# Подключаемся к ClickHouse через pandahouse

In [4]:
import pandahouse as ph

In [5]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

# 2.1 Очень усердные ученики

In [11]:
q = '''
    SELECT COUNT(st_id) AS student_amount
    FROM
    (
    SELECT st_id, SUM(correct) AS peases
    FROM default.peas
    GROUP BY st_id
    )
    WHERE peases >= 20
    '''

q_test = ph.read_clickhouse(query=q, connection=connection_default)
q_test

,student_amount
0,136


# 2.2 Оптимизация воронки

In [12]:
q_2 = '''
    SELECT a.test_grp, 
    ROUND(d.all_money / a.all_users, 2) AS ARPU, 
    ROUND(d.all_money / b.active_users, 2) AS ARPAU, 
    ROUND(d.paid_users * 100 / a.all_users, 2) AS purchase_CR,
    ROUND(d.paid_users * 100 / b.active_users, 2) AS active_CR,
    ROUND(e.paid_math_users * 100 / c.active_math_users, 2) AS math_CR
    FROM
        (
        SELECT a.test_grp, a.all_users, b.active_users, c.active_math_users, d.paid_users, d.all_money, e.paid_math_users
        FROM
            (
            SELECT test_grp, COUNT(st_id) as all_users 
            FROM
                (
                SELECT b.test_grp, a.st_id, a.subject, SUM(a.correct) as peases 
                FROM default.peas AS a
                LEFT JOIN default.studs AS b
                ON a.st_id = b.st_id
                GROUP BY a.st_id, a.subject, b.test_grp
                )
            GROUP BY test_grp
            ) AS a
            JOIN
            (
            SELECT test_grp, COUNT(st_id) as active_users 
            FROM
                (
                SELECT b.test_grp, a.st_id, a.subject, SUM(a.correct) as peases 
                FROM default.peas AS a
                LEFT JOIN default.studs AS b
                ON a.st_id = b.st_id
                GROUP BY a.st_id, a.subject, b.test_grp
                )
            WHERE peases > 10
            GROUP BY test_grp
            ) AS b 
            ON a.test_grp = b.test_grp
            JOIN
            (
            SELECT test_grp, COUNT(st_id) as active_math_users
            FROM
                (
                SELECT b.test_grp, a.st_id, a.subject, SUM(a.correct) as peases
                FROM default.peas AS a
                LEFT JOIN default.studs AS b
                ON a.st_id = b.st_id
                GROUP BY a.st_id, a.subject, b.test_grp
                )
            WHERE subject = 'Math' AND peases > 2
            GROUP BY test_grp
            ) AS c
            ON b.test_grp = c.test_grp
            JOIN
            (
            SELECT b.test_grp, COUNT(a.st_id) AS paid_users, SUM(a.money) AS all_money
            FROM default.final_project_check AS a                                      
            LEFT JOIN default.studs AS b                                               
            ON a.st_id = b.st_id
            GROUP BY b.test_grp
            ) AS d
            ON c.test_grp = d.test_grp
            JOIN
            (
            SELECT b.test_grp, COUNT(a.st_id) AS paid_math_users
            FROM default.final_project_check AS a
            LEFT JOIN default.studs AS b
            ON a.st_id = b.st_id
            WHERE subject = 'Math'
            GROUP BY b.test_grp
            ) AS e
            ON d.test_grp = e.test_grp
        )
    '''

q_2_test = ph.read_clickhouse(query=q_2, connection=connection_default)
q_2_test

,a.test_grp,ARPU,ARPAU,purchase_CR,active_CR,math_CR
0,control,5306.51,8765.82,6.90,11.39,16.33
1,pilot,13913.93,24963.24,17.62,31.62,38.10
